In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import time

In [ ]:
html=requests.get("https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador-del-30-de-junio-de-2022?idiom=es")
soup=bs(html.text,"html5lib")
texto=soup.find("div","article-body").text
texto=texto.split(':')
for i,dialogo in enumerate(texto):
  if texto[i-1][-7:] == 'OBRADOR':
    dialogo_amlo=texto[i]
    print(dialogo_amlo)
  elif texto[i-1][-8:] == 'PREGUNTA' and texto[i][-7:] == 'OBRADOR':
    pregunta_amlo=texto[i]
    respuesta_amlo=texto[i+1]

 Buenos días. Ánimo, que lo mejor es lo peor que se va a poner. Villa lo decía de otra manera.

Bueno, vamos a informar. Como todos los jueves, el licenciado Ricardo Mejía Berdeja va a informar sobre cómo se van presentando crímenes que conmueven a la sociedad, a la gente, y que requieren, como todos los asesinatos, atención, de que haya investigación, que no haya impunidad. Por eso, la sección se llama Cero Impunidad, con la idea muy clara de que el que comete un delito va a ser juzgado. Puede llevarnos tiempo en tanto se le encuentra, pero no hay impunidad para nadie, cero impunidad.

Entonces, vamos a escuchar al licenciado Mejía.

RICARDO MEJÍA BERDEJA, SUBSECRETARIO DE SEGURIDAD PÚBLICA DE LA SECRETARÍA DE SEGURIDAD Y PROTECCIÓN CIUDADANA (SSPC)
 Uno, dos, tres, cuatro, cinco, seis y siete, y ocho si nos alcanza.

PREGUNTA
 Pues yo pienso que es una conjunción de varios factores. Desde luego, el telón de fondo es la inconformidad de quienes no aceptan el que se haya hecho el aerop

In [ ]:
html=requests.get("https://www.gob.mx/presidencia/es/archivo/articulos?filter_origin=archive&idiom=es&order=DESC&page=1")
soup=bs(html.text,"html.parser")
textos=soup.find_all("a")
fechas_aux=soup.find_all("time")
fechas=[fecha.get("date").replace(u'\\"',u'') for fecha in fechas_aux]
for texto in textos:
  if "conferencia" in texto.get('href'):
    print(fechas)
    xd=texto.get("href").replace(u'\\"',u'').replace('/p','http://www.gob.mx/p')
    ht=requests.get(xd)
    sopa=bs(ht.text,"html5lib")

['2022-07-04', '2022-07-04', '2022-07-01', '2022-07-01', '2022-07-01', '2022-06-30', '2022-06-30', '2022-06-29', '2022-06-28']
['2022-07-04', '2022-07-04', '2022-07-01', '2022-07-01', '2022-07-01', '2022-06-30', '2022-06-30', '2022-06-29', '2022-06-28']
['2022-07-04', '2022-07-04', '2022-07-01', '2022-07-01', '2022-07-01', '2022-06-30', '2022-06-30', '2022-06-29', '2022-06-28']
['2022-07-04', '2022-07-04', '2022-07-01', '2022-07-01', '2022-07-01', '2022-06-30', '2022-06-30', '2022-06-29', '2022-06-28']


In [ ]:
def obtener_datos(num_p):
  df_dialogos=pd.DataFrame(columns=["Fecha","Dialogo"])
  df_preguntas=pd.DataFrame(columns=["Fecha","Pregunta","Respuesta"])
  for n in range(num_p):
    html=requests.get(f"https://www.gob.mx/presidencia/es/archivo/articulos?filter_origin=archive&idiom=es&order=DESC&page={str(n+1)}")
    sopa=bs(html.text,"html5lib")
    urls=sopa.find_all("a")
    fechas_aux=sopa.find_all("time")
    fechas=[fecha.get("date") for fecha in fechas_aux]
    for j,url in enumerate(urls):
      fecha=fechas[j] if j <9 else None
      if "conferencia" in url.get('href'):
        url=url.get("href").replace(u'\\"',u'').replace('/p','http://www.gob.mx/p')
        html=requests.get(url)
        soup=bs(html.text,"html5lib")
        conferencia=soup.find("div","article-body").text
        dialogos=conferencia.split(':')
        for i in range(len(dialogos)):
          if dialogos[i][-7:] == 'OBRADOR':
            dialogo_amlo=dialogos[i+1]
            df_dialogos=df_dialogos.append({"Fecha":fecha,"Dialogo":dialogo_amlo},ignore_index=True)
          elif (dialogos[i][-8:] == 'PREGUNTA' or dialogos[i][-12:] == 'INTERLOCUTOR' or dialogos[i][-12:] == 'INTERVENCIÓN') and dialogos[i+1][-7:] == 'OBRADOR':
            pregunta_amlo=dialogos[i+1]
            respuesta_amlo=dialogos[i+2]
            df_preguntas=df_preguntas.append({"Fecha":fecha,"Pregunta":pregunta_amlo,"Respuesta":respuesta_amlo},ignore_index=True)
    time.sleep(5)
  return df_preguntas,df_dialogos

In [ ]:
df1,df2=obtener_datos(1)
print(df1.tail(10))
print(df2.tail())

    Fecha                                           Pregunta  \
397  None   Los transportistas dicen que sí pueden llegar...   
398  None   Sobre si tenía fecha de cuándo vuelven a reto...   
399  None   Mi segundo tema, presidente, mi segundo tema ...   
400  None   Sobre las pensiones y todo esto, ¿todavía no ...   
401  None   Muchas gracias, presidente.\n\nPRESIDENTE AND...   
402  None   ¿Y tiene otras reuniones?\n\nPRESIDENTE ANDRÉ...   
403  None   ¿Con quién?\n\nPRESIDENTE ANDRÉS MANUEL LÓPEZ...   
404  None      Sí.\n\nPRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR   
405  None   ¿No le regaló su jersey?\n\nPRESIDENTE ANDRÉS...   
406  None   De Michoacán.\n\nPRESIDENTE ANDRÉS MANUEL LÓP...   

                                             Respuesta  
397   Sí, y lo estamos haciendo, lo estamos haciend...  
398   Ah, está por resolverse lo de los amparos, no...  
399  Bueno, yo creo que —no estaba yo informado— pe...  
400   No, no, no puedo todavía, pero a ver qué pode...  
401   Ya s